# The Workspace

The workspace pattern provides a shared, persistent file system where agents externalize artifacts too large for the context window. Agents see sandbox paths (`/workspace/...`) while actual files are stored in isolated directories per user and session.

In [1]:
from pathlib import PurePosixPath
import json

from agentic_patterns.core.workspace import (
    workspace_to_host_path,
    write_to_workspace,
    read_from_workspace,
    list_workspace_files,
    WorkspaceError,
)
from agentic_patterns.core.user_session import (
    set_user_session,
    get_user_id,
    get_session_id,
)
from agentic_patterns.core.config.config import WORKSPACE_DIR
from agentic_patterns.core.agents import get_agent, run_agent

## Identity Setup

The workspace resolves user/session identity from contextvars. Call `set_user_session()` once at the request boundary (middleware, MCP handler, etc.) and all downstream code picks it up automatically.

In [2]:
set_user_session("alice", "session_001")

print(f"User: {get_user_id()}")
print(f"Session: {get_session_id()}")
print(f"Workspace base: {WORKSPACE_DIR}")

User: alice
Session: session_001
Workspace base: /Users/kqrw311/workspace/aixplore/book_agentic_patterns/data/workspaces


## Path Translation

Helper functions convert between sandbox paths (what agents see) and host paths (actual filesystem). User/session identity is resolved automatically from contextvars.

In [3]:
sandbox_path = "/workspace/reports/analysis.json"
host_path = workspace_to_host_path(PurePosixPath(sandbox_path))

print(f"Agent sees:    {sandbox_path}")
print(f"Actual file:   {host_path}")

Agent sees:    /workspace/reports/analysis.json
Actual file:   /Users/kqrw311/workspace/aixplore/book_agentic_patterns/data/workspaces/alice/session_001/reports/analysis.json


## Tool Pattern: Write Large Output, Return Summary

When a tool produces large output, it writes the full result to the workspace and returns a concise summary with the file path. The agent's context stays small.

In [4]:
def analyze_dataset(query: str) -> str:
    """Analyze data and save results to workspace."""
    result = {
        "query": query,
        "row_count": 50000,
        "statistics": {"mean": 42.5, "std": 12.3, "min": 0.1, "max": 99.8},
        "data": [{"id": i, "value": i * 0.1} for i in range(1000)],
    }

    output_path = "/workspace/analysis/result.json"
    write_to_workspace(output_path, json.dumps(result, indent=2))

    return f"""Analysis complete. Rows: {result["row_count"]}, Mean: {result["statistics"]["mean"]}
Full results: {output_path}"""

In [5]:
summary = analyze_dataset("SELECT * FROM metrics")
print(summary)

# Verify file exists on disk
host_path = workspace_to_host_path(PurePosixPath("/workspace/analysis/result.json"))
print(f"\nFile size: {host_path.stat().st_size} bytes")

Analysis complete. Rows: 50000, Mean: 42.5
Full results: /workspace/analysis/result.json

File size: 54789 bytes


## Agent with Workspace Tools

Tools no longer need closures to capture context. Since user/session is in contextvars, tools just call workspace functions directly.

In [6]:
def search_data(query: str) -> str:
    """Search dataset and save results to workspace."""
    matches = [
        {"id": i, "name": f"item_{i}", "score": 0.9 - i * 0.01} for i in range(500)
    ]

    output_path = "/workspace/search_results.json"
    write_to_workspace(output_path, json.dumps(matches))

    return f"Found {len(matches)} matches. Top 3: {matches[:3]}. Full results: {output_path}"


def read_file(path: str) -> str:
    """Read a file from the workspace."""
    return read_from_workspace(path)


tools = [search_data, read_file]

In [7]:
agent = get_agent(tools=tools)

prompt = "Search for sensor data and tell me how many results were found."
agent_run, nodes = await run_agent(agent, prompt, verbose=True)

print(f"\nAnswer: {agent_run.result.output}")

Agent step: UserPromptNode(user_prompt='Search for sensor data and tell me how many results were found.', 
instructions_functions=[], system_prompts=(), system_prompt_functions=[], system_prompt_dynamic_functions={})

Agent step: ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content='Search for sensor data and tell me
how many results were found.', timestamp=datetime.datetime(2026, 2, 12, 13, 56, 16, 323032, 
tzinfo=datetime.timezone.utc))]))

Agent step: CallToolsNode(model_response=ModelResponse(parts=[TextPart(content="I'll search for sensor data and let
you know how many results are found."), ToolCallPart(tool_name='search_data', args='{"query": "sensor data"}', 
tool_call_id='toolu_vrtx_011gg9BhALxzEnYYpjiB5xbz')], usage=RequestUsage(input_tokens=639, output_tokens=71, 
details={'is_byok': False, 'reasoning_tokens': 0}), model_name='anthropic/claude-sonnet-4.5', 
timestamp=datetime.datetime(2026, 2, 12, 13, 56, 18, 315714, tzinfo=datetime.timezone.utc), provider_name='openai',
provider_url='https://openrouter.ai/api/v1/', provider_details={'finish_reason': 'tool_calls', 'timestamp': 
datetime.datetime(2026, 2, 12, 13, 56, 16, tzinfo=TzInfo(0))}, 
provider_response_id='gen-1770904576-8sWgfzbD877TQZnbyGTg', finish_reason='tool_call', 
run_id='7bfc9d5d-5e7f-4a66-b44e-b38b4380a5a8'))

Agent step: ModelRequestNode(request=ModelRequest(parts=[ToolReturnPart(tool_name='search_data', content="Found 500
matches. Top 3: [{'id': 0, 'name': 'item_0', 'score': 0.9}, {'id': 1, 'name': 'item_1', 'score': 0.89}, {'id': 2, 
'name': 'item_2', 'score': 0.88}]. Full results: /workspace/search_results.json", 
tool_call_id='toolu_vrtx_011gg9BhALxzEnYYpjiB5xbz', timestamp=datetime.datetime(2026, 2, 12, 13, 56, 18, 336787, 
tzinfo=datetime.timezone.utc))]))

Agent step: CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='The search for "sensor data" found 
**500 matches**. \n\nThe top 3 results are:\n1. item_0 (score: 0.9)\n2. item_1 (score: 0.89)\n3. item_2 (score: 
0.88)\n\nThe full results have been saved to `/workspace/search_results.json` if you\'d like to explore them 
further.')], usage=RequestUsage(input_tokens=820, output_tokens=98, details={'is_byok': False, 'reasoning_tokens': 
0}), model_name='anthropic/claude-sonnet-4.5', timestamp=datetime.datetime(2026, 2, 12, 13, 56, 20, 603427, 
tzinfo=datetime.timezone.utc), provider_name='openai', provider_url='https://openrouter.ai/api/v1/', 
provider_details={'finish_reason': 'stop', 'timestamp': datetime.datetime(2026, 2, 12, 13, 56, 18, 
tzinfo=TzInfo(0))}, provider_response_id='gen-1770904578-YRAliBMaMSuzxcWkJ0Yy', finish_reason='stop', 
run_id='7bfc9d5d-5e7f-4a66-b44e-b38b4380a5a8'))

Agent step: End(data=FinalResult(output='The search for "sensor data" found **500 matches**. \n\nThe top 3 results 
are:\n1. item_0 (score: 0.9)\n2. item_1 (score: 0.89)\n3. item_2 (score: 0.88)\n\nThe full results have been saved 
to `/workspace/search_results.json` if you\'d like to explore them further.'))


Answer: The search for "sensor data" found **500 matches**. 

The top 3 results are:
1. item_0 (score: 0.9)
2. item_1 (score: 0.89)
3. item_2 (score: 0.88)

The full results have been saved to `/workspace/search_results.json` if you'd like to explore them further.


In [8]:
# Full data persists in workspace
print("Files in workspace:")
for f in list_workspace_files("*.json"):
    host_path = workspace_to_host_path(PurePosixPath(f))
    print(f"  {f} ({host_path.stat().st_size} bytes)")

Files in workspace:
  /workspace/search_results.json (27717 bytes)
  /workspace/analysis/result.json (54789 bytes)


## User Isolation

Each user/session gets an isolated directory. The identity extractor determines which workspace to use based on the request context.

In [9]:
# Switch to Bob's context
set_user_session("bob", "session_001")
write_to_workspace("/workspace/secret.txt", "Bob's private data")
bob_path = workspace_to_host_path(PurePosixPath("/workspace/secret.txt"))

# Switch back to Alice
set_user_session("alice", "session_001")
alice_path = workspace_to_host_path(PurePosixPath("/workspace/secret.txt"))

print(f"Bob's file:   {bob_path}")
print(f"Alice's file: {alice_path}")

Bob's file:   /Users/kqrw311/workspace/aixplore/book_agentic_patterns/data/workspaces/bob/session_001/secret.txt
Alice's file: /Users/kqrw311/workspace/aixplore/book_agentic_patterns/data/workspaces/alice/session_001/secret.txt


## Security: Path Traversal Prevention

In [10]:
try:
    workspace_to_host_path(PurePosixPath("/workspace/../../../etc/passwd"))
except WorkspaceError as e:
    print(f"Blocked: {e}")

try:
    workspace_to_host_path(PurePosixPath("/etc/passwd"))
except WorkspaceError as e:
    print(f"Blocked: {e}")

Blocked: Path traversal not allowed
Blocked: Invalid sandbox path: must start with /workspace
